In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import re
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Загрузка модели и токенизатора
model = AutoModelForSequenceClassification.from_pretrained("./saved_model")
tokenizer = AutoTokenizer.from_pretrained("./saved_model")

# Используем устройство CUDA, если доступно
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [8]:
# Функция нормализации текста
def normalize_text(text):
    text = text.strip().lower()
    text = re.sub(r'\s+', ' ', text)  # Удаление лишних пробелов
    text = re.sub(r'[^\w\s]', '', text)  # Удаление пунктуации и неалфавитных символов
    return text


In [14]:
# Ввод заголовка и описания видео
title = input("Введите заголовок видео: ")
description = input("Введите описание видео: ")

# Объединение и нормализация текста
input_text = title + " " + description
normalized_text = normalize_text(input_text)

# Токенизация
encoding = tokenizer(normalized_text, return_tensors='pt', truncation=True, padding=True, max_length=256)
encoding = {k: v.to(device) for k, v in encoding.items()}

# Предсказание
with torch.no_grad():
    outputs = model(**encoding)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)

# Загрузка кодировщика меток, использованного для обучения
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('./saved_model/label_encoder_classes.npy', allow_pickle=True)

# Вывод тегов с вероятностями
top_k = 5  # Количество выводимых тегов
probabilities, indices = torch.topk(probabilities, top_k, dim=1)

print("Предсказанные теги с вероятностями:")
for prob, idx in zip(probabilities[0], indices[0]):
    tag = label_encoder.inverse_transform([idx.item()])[0]
    print(f"{tag}: {prob.item():.4f}")


Введите заголовок видео:  TikTok Дайджест | Выпуск 3
Введите описание видео:  Встречайте свежий выпуск с Настей Витоновой. Свежие сплетни, актуальные тренды, интересные новости - смотрите, внимайте, обожайте!


Предсказанные теги с вероятностями:
Массовая культура: 0.9967
Массовая культура: Отношения знаменитостей: 0.0007
Музыка и аудио: Комедия и стендап (Музыка и аудио): 0.0002
Массовая культура: Юмор и сатира: 0.0002
Хобби и интересы: 0.0002
